# GPS

The rocket has a Conexant Jupiter GPS, a 12-channel OEM receiver based on the Zodiac chipset. It was set to send 1 Hz binary data.

## Altitude

We can compare the GPS reported altitude to the IMU integration.

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var, argmax, add, average, abs
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')


time = []
gps_h = []
climb = []
with open("../telemetry.csv", 'r') as f_in:
    for line, text in enumerate(f_in.readlines()):
        if line > 1:
            li = text.split(',')
            try:
                height = float(li[39])
                c = float(li[39+5])
            except ValueError:
                continue
            time.append(float(li[0]))
            gps_h.append(height)
            climb.append(c)

# Jump to the interesting part of the flight
cuttoff = -75
time = time[cuttoff:]
gps_h = gps_h[cuttoff:]
climb = climb[cuttoff:]

# set MET=0
with open("launch-time.csv", 'r') as fin:
    liftoff = float(fin.read())
time = subtract(time, liftoff)

# units
gps_h = divide(gps_h, 100.0) # meters
gps_h = subtract(gps_h, 1390.0) # AGL
climb = divide(climb, 100.0) # meters

In [ ]:
columns = loadtxt("uncalibrated_integrated_altitude.csv", delimiter=',', unpack=True)
alt_time = columns[0]
imualt   = columns[1]

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"GPS Reported Height AGL")
plt.ylabel(r"GPS Height AGL [km]")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.plot(alt_time, divide(imualt, 1000.0), 'k-', alpha=0.2, label="IMU Integrated Altitude")
plt.plot(time, divide(gps_h, 1000.0), '-o', alpha=0.75, ms=3, markeredgewidth=0, label="GPS Height")
plt.xlim([-4,70])
ax1.legend(loc=1)
plt.show()

There does appear to be a time lag for some reason between the two.

In [ ]:
last_gps_time = time[-1]
idx = (abs(alt_time-last_gps_time)).argmin()

print """The maximum GPS altitude recorded was %0.2f kilomters. (IMU: %0.2f km).
Though there was some forcing in the IMU calibration to match this numer. The final
data point from the gps is %0.0f meters. At that time the integrated IMU reports
%0.0f meters, a close match!
""" % (max(gps_h)/1000.0, max(imualt)/1000.0, gps_h[-1], imualt[idx])

## Velocity

A similar story for velocity. There is also good agreement here, again with a time lag. Also the peak of the velocity curve appears missing from the GPS. This might have something to do with the ability to do fixes under high-g loads.

In [ ]:
columns = loadtxt("uncalibrated_integrated_velocity.csv", delimiter=',', unpack=True)
veltime = columns[0]
imuvel   = columns[1]

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"GPS Reported Climb Rate")
plt.ylabel(r"Climb Rate [m/s]")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.plot(veltime, imuvel, 'k-', alpha=0.2, label="IMU Integrated Velocity")
plt.plot(time, climb, '-o', alpha=0.75, ms=3, markeredgewidth=0, label="GPS Vertical Climb Rate")
plt.xlim([-4,70])
ax1.legend(loc=1)
plt.show()

In [ ]:
print """We continue to get IMU data for %0.2f seconds after the last GPS data point.
""" % (veltime[-1] - time[-1])